In [1]:
# Escrever no ficheiro
#with open('formatted_imgs.txt', 'w') as f:
    
    # OPÇAO 1
 #   f.write(str(ls))
    
    # OPÇAO 2
  #  for i in ls[:-1]:
   #    f.write(str(i) + ',\n')
    #f.write(str(ls[-1]) + '\n')  
    
#f.close()

In [2]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# Import relevant libraries
import numpy as np
from numpy import asarray
from PIL import Image
import os

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
# from kt_utils import *
from keras.losses import BinaryCrossentropy

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# Training Images


In [ ]:
directory = 'faces_train'
ls = []
label_ls = []
i = 0
for filename in os.listdir(directory):
    i += 1
    if (i % 1000 == 0):
        print(i)
    img = Image.open(directory + '/' + filename)
    data = asarray(img)
    ls.append(data)
    label_ls.append(1)

directory = 'no_faces_train'
for filename in os.listdir(directory):
    i += 1
    if (i % 1000 == 0):
        print(i)
    img = Image.open(directory + '/' + filename)
    data = asarray(img)
    ls.append(data)
    label_ls.append(0)
    
imgs_train = np.array(ls, dtype='uint8')
label_train = np.array([label_ls])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


# Testing images

In [ ]:
directory = 'faces_test'
ls = []
label_ls = []
i = 0
for filename in os.listdir(directory):
    i += 1
    if (i % 1000 == 0):
        print(i)
    img = Image.open(directory + '/' + filename)
    data = asarray(img)
    ls.append(data)
    label_ls.append(1)

directory = 'no_faces_test'
for filename in os.listdir(directory):
    i += 1
    if (i % 1000 == 0):
        print(i)
    img = Image.open(directory + '/' + filename)
    data = asarray(img)
    ls.append(data)
    label_ls.append(0)
    
imgs_test = np.array(ls, dtype='uint8')
label_test = np.array([label_ls])

In [ ]:
# Normalize image vectors
#imgs_train_norm = imgs_train / 255.
#imgs_test_norm = imgs_test / 255.

# Reshape
label_train_T = label_train.T
label_test_T = label_test.T

print ("Number of training examples = " + str(imgs_train.shape[0]))
print ("Number of testing examples = " + str(imgs_test.shape[0]))
print ("Images train shape: " + str(imgs_train.shape))
print ("Labels train shape: " + str(label_train_T.shape))
print ("Images test shape: " + str(imgs_test.shape))
print ("Labels test shape: " + str(label_test_T.shape))

In [ ]:
# HappyModel

def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    # Use the suggested model in the text above to get started, and run through the whole
    # exercise once. Then come back and add more BLOCKS. 
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)
    
    # BLOCK 1: CONV -> BN -> RELU -> MAXP
    num_filters = 32
    kernel_size = (7, 7)
    stride = (1, 1)
    X = Conv2D(num_filters, kernel_size, strides = stride, name = 'conv0')(X)

    # BLOCK 2: CONV -> BN -> RELU -> MAXP
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
        
    # BLOCK 3: CONV -> BN -> RELU -> MAXP
    X = Activation('relu')(X)
    
    # BLOCK 4: CONV -> BN -> RELU -> MAXP
    max_pool_size = (2, 2)
    X = MaxPooling2D(max_pool_size, name='max_pool')(X)
       
        
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model

In [ ]:
happyModel = HappyModel(imgs_train.shape[1:])

In [ ]:
happyModel.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=["accuracy"])

In [ ]:
happyModel.fit(imgs_train, label_train_T, 
              batch_size = 16,
              epochs = 5,
              validation_data = (imgs_test, label_test_T))

In [ ]:
# happyModel.fit(x = imgs_test_norm, y = label_test_T, batch_size = 32, epochs = 4)

In [ ]:
preds = happyModel.evaluate(x = imgs_test, y = label_test_T)

print()
print ("Loss = " + str(preds[0]))
print ("Train Accuracy = " + str(preds[1]))

In [ ]:
#preds = happyModel.evaluate(x = imgs_test_norm, y = label_test_T)

#print()
#print ("Loss = " + str(preds[0]))
#print ("Test Accuracy = " + str(preds[1]))

In [ ]:
# plots the model in a nice layout and save it as ".png"
#plot_model(happyModel, to_file='HappyModel.png')

In [ ]:
img_path = 'faces_train/1_97.jpg'
img = image.load_img(img_path, target_size=(256, 256))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))